In [1]:
import os
import numpy as np
import pandas as pd

import riskflow_jupyter

In [2]:
rundate = '2022-07-07'
if os.name=='nt':
    path = os.path.join('U:\\CVA_JSON', rundate)
    path_transform={
        '\\\\ICMJHBMVDROPPRD\\AdaptiveAnalytics\\Inbound\\MarketData':
        '\\\\ICMJHBMVDROPUAT\\AdaptiveAnalytics\\Inbound\\MarketData'}
else:
    path = os.path.join('/media/vretiel/3EFA4BCDFA4B7FDF/Media/Data/crstal/CVA_JSON', rundate)
    path_transform={
            '//ICMJHBMVDROPPRD/AdaptiveAnalytics/Inbound/MarketData': 
            '/media/vretiel/3EFA4BCDFA4B7FDF/Media/Data/crstal/CVA_JSON'}
    
cx = riskflow_jupyter.rf.Context(
    path_transform=path_transform,
    file_transform={
        'CVAMarketData_Calibrated.dat': 'CVAMarketData_Calibrated_New.json',
        'MarketData.dat': 'MarketData.json'
    })

In [3]:
from ipywidgets import Label, HBox, VBox
from riskflow_widgets import Table

In [4]:
# cx.load_json(os.path.join(path, 'InputAAJ_CrB_Russellstone_Treasury_ISDA.json'))
cx.load_json(os.path.join(path, 'InputAAJ_CrB_BNP_Paribas__Paris__ISDA.json'))


In [ ]:
import ipywidgets
dir(ipywidgets)

In [ ]:
factor = riskflow_jupyter.rf.utils.Factor('FXVol',('GBP','ZAR'))
vol = riskflow_jupyter.rf.riskfactors.construct_factor( factor,
    cx.current_cfg.params['Price Factors'],
    cx.current_cfg.params['Price Factor Interpolation']
)

In [ ]:
import k3d
import json


def to_json(string):
    """converts a string to json - skips the whitespace for a smaller output"""
    return json.dumps(string, separators=(',', ':'))


def load_table_from_vol(vol):
    sparse_matrix = {}
    for k in vol.param['Surface'].array:
        sparse_matrix.setdefault(k[0], {}).setdefault(k[1], k[2])

    df = pd.DataFrame(sparse_matrix).sort_index(0, 1)

    table = np.vstack([[0.0]+df.columns.values.round(5).tolist(), 
               np.hstack([df.index.values.round(5).reshape(-1,1).tolist(),
                          df.round(5).replace({np.nan:None})])]).tolist()
    
    return table


class Three(HBox):
    def __init__(self, description):
        self.description = Label(value=description)        
        self.plot = k3d.plot(
            axes=['log(moneyness)', 'expiry','vol(\%)'],
            #menu_visibility=False,
            camera_rotate_speed=3.0
            )
        self.mesh = None
        self.data = Table(description='', settings=to_json({
            'width':500, 'height':300, 'contextMenu':True, 'minSpareRows':1, 'minSpareCols':1
        }))                
        
        super().__init__(
            children=[
                self.description,
                VBox(children=[self.plot, self.data])
            ]
        )
        
    def observe(self, handler, prop, type='change'):
        def make_func():
            def handle_draw(change):
                handler(change)
                self.update_plot(json.loads(change['new']))
            return handle_draw
        
        if self.children:
            self.data.observe(make_func(), prop, type)
        else:
            super().observe(handler, prop, type)
        
    @staticmethod
    def make_faces_vectorized1(Nr,Nc):

        out = np.empty((Nr-1,Nc-1,2,3),dtype=int)

        r = np.arange(Nr*Nc).reshape(Nr,Nc)

        out[:,:, 0,0] = r[:-1,:-1]
        out[:,:, 1,0] = r[:-1,1:]
        out[:,:, 0,1] = r[:-1,1:]

        out[:,:, 1,1] = r[1:,1:]
        out[:,:, :,2] = r[1:,:-1,None]

        out.shape =(-1,3)
        return out
    
    def update_plot(self, json_list):
        e, moneyness, vol = Three.interpolate_surface(json_list)
        scale = 2/np.log(2)
        m = scale*np.log(moneyness)
        v = vol*100
        
        U, V = np.meshgrid(m, e)
        vertices = np.dstack([U, V, v]).reshape(-1,3)
        indices = Three.make_faces_vectorized1(*v.shape)
        if self.mesh is None:
            self.mesh = k3d.mesh(
                vertices.astype(np.float32), indices.astype(np.uint32), flat_shading=False, attribute=v,
                side='double', color_map=k3d.basic_color_maps.Reds, color_range=[v.min(), v.max()]
            )
            self.plot+=self.mesh
        else:
            self.mesh.vertices = vertices
            self.mesh.indices = indices
        
    @staticmethod
    def interpolate_surface(json_list):
        moneyness = json_list[0][1:]
        e = []
        for p in json_list[1:]:
            e.extend([[m, p[0], v] for m,v in zip(moneyness, p[1:]) if v is not None])

        surface = np.array(e)
        expiry = [p[0] for p in json_list[1:]]
        return expiry, moneyness, np.array([np.interp(
            moneyness, surface[surface[:, 1] == x][:, 0], surface[surface[:, 1] == x][:, 2]) for x in expiry])

    
    @property
    def value(self):        
        return self.data.value
    
    # a setter function
    @value.setter
    def value(self, json_list):
        self.data.value = to_json(json_list)
        self.update_plot(json_list)
        

In [ ]:
j=load_table_from_vol(vol)
j
#np.isnan(None)

In [ ]:
# g = np.vstack([x[0], heights]).tolist()
t = Three(description='Test' )
t.observe(lambda x:x['new'], 'value')


In [ ]:
# t.data.col_headers = ['x/y']+x[0].tolist()
t.value = j

In [ ]:
t.data.settings

In [ ]:
t

In [ ]:
t.mesh.attribute

In [ ]:
from ipyfilechooser import FileChooser

# Create and display a FileChooser widget
fc = FileChooser('/home/vretiel/miniconda3/')
display(fc)

In [ ]:
fc.selected_filename

In [ ]:
cx

In [5]:
pp = riskflow_jupyter.PortfolioPage(cx.current_cfg)

In [6]:
pp.main_container

In [7]:
rfp = riskflow_jupyter.RiskFactorsPage(cx.current_cfg)

In [8]:
rfp.main_container


/home/vretiel/miniconda3/lib/python3.8/site-packages/traittypes/traittypes.py:97: UserWarning: Given trait value dtype "float64" does not match required type "float32". A coerced copy has been created.
  warnings.warn(


KeyError: 0

In [ ]:
rfp.tree.checked


In [ ]:
a='InterestYieldVol.CAD_SMILE_ICE'

c= getattr(riskflow_jupyter.rf.riskfactors, riskflow_jupyter.rf.utils.check_rate_name(a)[0])

In [ ]:
c

In [ ]:
riskflow_jupyter.CalculationPage(cx.current_cfg)

In [ ]:
%debug

> /home/vretiel/PycharmProjects/riskflow_widgets/riskflow_widgets/widgets.py(151)interpolate_surface()
    149     @staticmethod
    150     def interpolate_surface(json_list):
--> 151         moneyness = json_list[0][1:]
    152         e = []
    153         for p in json_list[1:]:

ipdb> u
> /home/vretiel/PycharmProjects/riskflow_widgets/riskflow_widgets/widgets.py(162)update_plot()
    160 
    161     def update_plot(self, json_list):
--> 162         e, moneyness, vol = Three.interpolate_surface(json_list)
    163         scale = 2 / np.log(2)
    164         m = scale * np.log(moneyness)

ipdb> json_list
{'1.0': [[0.0, -0.025, -0.02, -0.015, -0.01, -0.0075, -0.0025, -0.0, 0.0025, 0.0075, 0.01, 0.015, 0.02, 0.025], [0.01918, 0.45417, 0.42918, 0.41331, 0.39677, 0.3885, 0.37316, 0.36694, 0.3626, 0.36321, 0.36998, 0.43102, 0.63267, 0.88848], [0.03836, 0.44123, 0.42635, 0.41046, 0.3939, 0.38562, 0.37024, 0.364, 0.35962, 0.36007, 0.3667, 0.39532, 0.44223, 0.61989], [0.08493, 0.43436, 0

ipdb> json_list.keys()
dict_keys(['1.0', '2.0', '3.0', '5.0', '10.0', '20.0', '30.0'])
